In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *


In [2]:
spark = SparkSession.builder.appName('StudentMarks').getOrCreate()

In [3]:
# Create the Student DataFrame
student_data = [
    (1, "Steve"),
    (2, "David"),
    (3, "Aryan")
]
student_df = spark.createDataFrame(student_data, ["Student_id", "Student_name"])

# Create the Subject DataFrame
subject_data = [
    (1, "pyspark", 90),
    (1, "sql", 100),
    (2, "sql", 70),
    (2, "pyspark", 60),
    (3, "sql", 30),
    (3, "pyspark", 20)
]
subject_df = spark.createDataFrame(subject_data, ["Student_id", "Subject_Name", "Marks"])

In [4]:
subject_df.show()

student_df.show()

+----------+------------+-----+
|Student_id|Subject_Name|Marks|
+----------+------------+-----+
|         1|     pyspark|   90|
|         1|         sql|  100|
|         2|         sql|   70|
|         2|     pyspark|   60|
|         3|         sql|   30|
|         3|     pyspark|   20|
+----------+------------+-----+

+----------+------------+
|Student_id|Student_name|
+----------+------------+
|         1|       Steve|
|         2|       David|
|         3|       Aryan|
+----------+------------+



In [5]:
join_df = student_df.join(subject_df, on = 'Student_id', how='Inner')



In [6]:
join_df.show()

+----------+------------+------------+-----+
|Student_id|Student_name|Subject_Name|Marks|
+----------+------------+------------+-----+
|         1|       Steve|     pyspark|   90|
|         1|       Steve|         sql|  100|
|         2|       David|         sql|   70|
|         2|       David|     pyspark|   60|
|         3|       Aryan|         sql|   30|
|         3|       Aryan|     pyspark|   20|
+----------+------------+------------+-----+



In [7]:
join_df.groupBy(col('Student_id'),col('student_name')).agg

# Calculate the percentage and result
result_df = join_df.groupBy("Student_id", "Student_name") \
    .agg(
        (sum("Marks") / (count("Subject_Name") * 100)).alias("Percentage"),
        when(sum("Marks") / (count("Subject_Name") * 100) >= 70, "Distinction")
        .when(sum("Marks") / (count("Subject_Name") * 100) >= 60, "First Class")
        .when(sum("Marks") / (count("Subject_Name") * 100) >= 50, "Second Class")
        .when(sum("Marks") / (count("Subject_Name") * 100) >= 40, "Third Class")
        .otherwise("Fail").alias("Result")
    )

result_df.show()



+----------+------------+----------+------+
|Student_id|Student_name|Percentage|Result|
+----------+------------+----------+------+
|         1|       Steve|      0.95|  Fail|
|         2|       David|      0.65|  Fail|
|         3|       Aryan|      0.25|  Fail|
+----------+------------+----------+------+

